In [1]:
import os
import sys
import plotly.express as px
from glob import glob
import numpy as np
import pandas as pd
from grader import notebook_to_module, grade_assignment, extract_assignment, get_students
from configparser import ConfigParser
import tqdm

In [2]:
root = os.path.abspath(os.path.join('..', '..', '..'))
grading_repo = os.path.join(root, 'cs-for-psych', 'assignments', 'autograder')
public_checks_dir = os.path.join(root, 'cs-for-psych', 'assignments')
private_checks_dir = os.path.join(root, 'teaching-tools', 'cs-for-psych', 'assignments')

In [3]:
#set up git
gitconfig = ConfigParser()
gitconfig.read(os.path.join(private_checks_dir, 'autograder', 'git.ini'))

os.system(f"git config --global user.name '{gitconfig['github']['user.name']}'");
os.system(f"git config --global user.email '{gitconfig['github']['user.email']}'");

In [4]:
#modify for each assignment
submissions_dir = os.path.join(root, 'Desktop', 'n-queens-01-06-2021-02-11-08')

In [5]:
assignment = extract_assignment(submissions_dir)
public_rubric = os.path.join(public_checks_dir, assignment, 'public_rubric.xls')
private_rubric = os.path.join(private_checks_dir, assignment, 'private_rubric.xls')

In [6]:
rubrics = [public_rubric, private_rubric]

In [7]:
students = get_students(submissions_dir)

In [8]:
def autograde_notebooks(submissions_dir, student):        
    def grade_notebook(notebook_fname, outfile=None, quiet=False):
        basedir, notebook = os.path.split(notebook_fname)        
        target = notebook[:-6]
        
        bad_chars = [' ', '-', ',', '!', '?', '.']
        modified_target = target
        for b in bad_chars:
            if b in modified_target:
                modified_target = modified_target.replace(b, '_')
        
        print(f'grading: {modified_target}.ipynb')
        clean_up = False
        if modified_target != target:
            if os.path.exists(os.path.join(basedir, modified_target)):
                clean_up = True
                os.rename(os.path.join(basedir, modified_target + '.ipynb'), os.path.join(basedir, modified_target + '.ipynb.BACKUP'))
            os.rename(os.path.join(basedir, target + '.ipynb'), os.path.join(basedir, modified_target + '.ipynb'))
        
        if 'submission' in sys.modules:
            print('pop!')

            sys.modules.pop('submission')
        
        submission = notebook_to_module(modified_target)
        score = grade_assignment(submission, rubrics, quiet=quiet, outfile=outfile)
                        
        if modified_target != target:
            os.rename(os.path.join(basedir, modified_target + '.ipynb'), os.path.join(basedir, target + '.ipynb'))
        
        if clean_up:
            os.rename(os.path.join(basedir, modified_target + '.ipynb.BACKUP'), os.path.join(basedir, modified_target + '.ipynb'))
        
        return score
    
    start_dir = os.getcwd()    
    mydir = os.path.join(submissions_dir, student)
    
    os.chdir(mydir)
    notebooks = glob(os.path.join('.', '*.ipynb'))
    grades = [grade_notebook(n, quiet=True) for n in notebooks]
    
    best_notebook = notebooks[np.where(np.array(grades) == np.max(grades))[0][0]]
    best_grade = grade_notebook(best_notebook, quiet=True, outfile='report.txt')    
    os.chdir(start_dir)
    return best_grade

In [9]:
grades = tqdm([autograde_notebooks(submissions_dir, s) for s in students])

grading: n_queens.ipynb
importing Jupyter notebook from n_queens.ipynb
-|0|1|2|3|4|5|6|7|
0|*|-|-|-|-|-|-|-|
1|-|-|-|-|-|-|*|-|
2|-|-|-|-|*|-|-|-|
3|-|-|-|-|-|-|-|*|
4|-|*|-|-|-|-|-|-|
5|-|-|-|*|-|-|-|-|
6|-|-|-|-|-|*|-|-|
7|-|-|*|-|-|-|-|-|
grading: n_queens.ipynb
importing Jupyter notebook from n_queens.ipynb
-|0|1|2|3|4|5|6|7|
0|*|-|-|-|-|-|-|-|
1|-|-|-|-|-|-|*|-|
2|-|-|-|-|*|-|-|-|
3|-|-|-|-|-|-|-|*|
4|-|*|-|-|-|-|-|-|
5|-|-|-|*|-|-|-|-|
6|-|-|-|-|-|*|-|-|
7|-|-|*|-|-|-|-|-|
grading: n_queens.ipynb
importing Jupyter notebook from n_queens.ipynb


AssertionError: something is wrong...

In [ ]:
grades = pd.DataFrame(zip(students, grades), columns=['Student', 'Score']).set_index('Student')

In [ ]:
fig = px.histogram(grades, x='Score', range_x=[0, 100], nbins=10, histnorm='percent');
fig.update_layout(yaxis_title='Percentage of students')

In [ ]:
#raise Exception('everything below this point is broken, and potentially harmful')

In [ ]:
def commit_and_push(submissions_dir, students, debug=True):
    feedback_branch = 'feedback'
    assignment = extract_assignment(submissions_dir)
    
    def commit_and_push_single_student(s, debug=True):
        mydir = os.path.join(submissions_dir, s)
        os.chdir(mydir)
        
        if debug:
            token = '<TOKEN>'
        else:
            token = gitconfig['github']['token']
        
        cmds = [f"git remote add {s} https://{token}@github.com/ContextLab/{assignment}-{s}.git",
                f'git fetch {s}',
                f'git checkout -b {feedback_branch}',
                'git add report.txt', 
                'git commit -a -m "added autograder report"',
                f'git push {s} HEAD:{feedback_branch}']
        if debug:
            print(f'Simulating commands (running from directory: {mydir})')
            [print('\t' + c) for c in cmds]
            print('\n')
        else:
            [os.system(c) for c in cmds]
    
    start_dir = os.getcwd()
    tqdm([commit_and_push_single_student(s, debug=debug) for s in students])
    os.chdir(start_dir)

In [ ]:
commit_and_push(submissions_dir, students, debug=False)